<h1 style="background-color:#1c6ce6;
           font-family:segoe ui;
                color:#FFF9ED;
           font-size:300%;
           text-align:center;
           border-radius:10px 10px;"> 
🗺️ תהליך הסבת שכבת תיקוני 97 ב' לבנק"ל מודרני 🗺️ 

In [ ]:
print(' Amendment 97B borders:')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
ספריות 📚

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import subprocess
import numpy as np
import pandas as pd
import datetime as dt
import geopandas as gpd
from configs import CNFG
import dask_geopandas as daskgpd
from helpers import assign_ParcelName

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">קריאת נתונים 📑 

In [ ]:
print('   Reading inputs')

<h1 style="text-align:right;font-size:100%">
טבלת ניהול תהליכים של הסדר קרקעות

In [ ]:
hek_df_cols = ['מספר פניה','תאריך בקשה','גוש','תת גוש','חלקה','תאריך שליחה לפקיד הסדר','תאריך אישור הממונה על המרשם','תאריך אישור הרשם',' שלב עבודה נוכחי','תאריך התחלה','תאריך סיום בפועל']
date_cols = ['תאריך בקשה', 'תאריך שליחה לפקיד הסדר', 'תאריך אישור הממונה על המרשם', 'תאריך אישור הרשם']

hek_df = pd.read_csv(CNFG.Local + '97b_nuhil_tahalichim.csv', usecols = hek_df_cols, encoding = 'cp1255', parse_dates = date_cols)

hek_df.sort_values('מספר פניה', inplace=True)


del hek_df_cols
print('       input 1/5 Read')

<h1 style="text-align:right;font-size:100%">
'שכבת תיקוני 97ב

In [ ]:
amendment97B_cols = ['MISPAR_PNIA', 'GUSH_NUM', 'GUSH_SUFFIX', 'PARCELS', 'geometry']

amendment97B_df = gpd.read_file(CNFG.CADSDE + 'Tikun_Hesder_97B.gdb', layer = 'Tikun_Hesder_97B', include_fields = amendment97B_cols)

del amendment97B_cols
print('       input 2/5 Read')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות קיימות

In [ ]:
parcels_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

parcels = daskgpd.read_file(CNFG.Arcstorm + 'PARCEL_ALL.gdb', layer = 'PARCEL_ALL', columns = parcels_cols, npartitions = 8).compute()

parcels = parcels.to_crs(2039)

del parcels_cols
print('       input 3/5 Read')

<h1 style="text-align:right;font-size:100%">
שכבת חלקות היסטוריות

In [ ]:
parcels_history_cols = ['PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX', 'geometry']

parcels_history = daskgpd.read_file(CNFG.CADSDE + 'PARCEL_HISTORY_HASAVA.gdb', layer = 'PARCEL_HISTORY_HASAVA', columns = parcels_history_cols, npartitions = 8).compute()

del parcels_history_cols
print('       input 4/5 Read')

<h1 style="text-align:right;font-size:100%">
טבלת מספרי זיהוי של גושים

In [ ]:
Blocks = pd.read_csv(CNFG.CSVPath + 'BlockUniqueIDs.csv', usecols = ['Name', 'GlobalID', 'IsTax'])
Blocks = Blocks[Blocks['IsTax'] == 0].drop(columns = 'IsTax')
print('       input 5/5 Read')

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;">
עיבוד נתונים 🛠️

In [ ]:
print('   Cleaning data')

<h1 style="text-align:right;font-size:125%">
:יצירת שכבת חלקות כללית
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש לזיהוי חלקה לפי מספרה, מספר הגוש, ומסתר התת גוש כשדה טקסט
<h1 style="text-align:right;font-size:100%">
יצירת שכבה חדשה
<h1 style="text-align:right;font-size:100%">
הסרת כפילויות

In [ ]:
parcels = parcels[(parcels['PARCEL'] != 0) & (parcels['GUSH_NUM'] != 0)]
parcels_history = parcels_history[(parcels_history['PARCEL'] != 0) & (parcels_history['GUSH_NUM'] != 0)]

parcels = assign_ParcelName(parcels, 'PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX')
parcels_history = assign_ParcelName(parcels_history, 'PARCEL', 'GUSH_NUM', 'GUSH_SUFFIX')

parcels_deep = pd.concat([parcels, parcels_history], ignore_index = False)\
                 .sort_values('ParcelName', ignore_index = True)\
                 .drop_duplicates(keep = 'first')

del [parcels, parcels_history]

<h1 style="text-align:right;font-size:125%">
:עיבוד טבלת ניהול תהליכים של הסדר קרקעות
<h1 style="text-align:right;font-size:100%">
הסרת רשומות עם ערכי חלקה לא חוקיים (אלו מקרים של בדיקה טבנית במערכת ניהול תהליכים)
<h1 style="text-align:right;font-size:100%">
יצירת שדה לסטטוס נוכחי ולתאריך שלו
<h1 style="text-align:right;font-size:100%">
יצירת שדה של כלל חלקות לפי מספר פניה
<h1 style="text-align:right;font-size:100%">
השלמת ערכים חסרים לשדה תת-גוש
<h1 style="text-align:right;font-size:100%">
יצירת שדה חדש לזיהוי חלקה לפי מספרה, מספר הגוש, ומסתר התת גוש כשדה טקסט

In [ ]:
print('   Data conversion')

In [ ]:
hek_df = hek_df[~hek_df['חלקה'].isin([9999.0, 999.0])]

hek_df['תאריך בקשה'] = pd.to_datetime(hek_df['תאריך בקשה'], format = '%d/%m/%Y')
hek_df['recent_date'] = hek_df[date_cols].max(axis = 1)
s4 = hek_df['recent_date'].eq(hek_df['תאריך אישור הרשם'])
s3 = hek_df['recent_date'].eq(hek_df['תאריך אישור הממונה על המרשם'])
s2 = hek_df['recent_date'].eq(hek_df['תאריך שליחה לפקיד הסדר'])
s1 = hek_df['recent_date'].eq(hek_df['תאריך בקשה'])

hek_df['status_name'] = np.select(condlist = (s4, s3, s2, s1),
                                  choicelist = ('אישור הרשם', 'אישור הממונה על הרשם', 'שליחה לפקיד הסדר', 'בקשה'),
                                  default = 'בקשה')


hek_status_clf = {'בקשה':14,
                  'שליחה לפקיד הסדר':16,
                  'אישור הממונה על הרשם':13,
                  'אישור הרשם':13}

hek_df['status'] = hek_df['status_name'].map(hek_status_clf)
hek_df['status'] = hek_df['status'].astype('Int32')


hek_df['חלקות'] = hek_df.astype(str).groupby(['מספר פניה'])['חלקה'].transform(','.join)


hek_df['תת גוש'] = hek_df['תת גוש'].fillna(0).astype(int)

hek_df.rename(columns = {'חלקה' : 'ParcelNumber', 'גוש' : 'BlockNumber', 'תת גוש' : 'SubBlockNumber'}, inplace = True)
hek_df = assign_ParcelName(hek_df, 'ParcelNumber', 'BlockNumber', 'SubBlockNumber', drop=False)

<h1 style="text-align:right;font-size:125%">
:יצירת שכבה של טבלת ניהול תהליכים
<h1 style="text-align:right;font-size:100%">
מיזוג הטבלה עם שכבת החלקות הכוללת שנוצרה בסעיף קודם
<h1 style="text-align:right;font-size:100%">
קיבוץ מרחבי לפי מספר פניה

In [ ]:
hek_df = hek_df.merge(parcels_deep[['geometry', 'ParcelName']], on = 'ParcelName', how = 'left')

hek_gdf = gpd.GeoDataFrame(hek_df, geometry = 'geometry', crs = 2039)

hek_gdf_dissolve = hek_gdf.dissolve(by='מספר פניה', aggfunc='max', as_index=False, dropna=False)

del [hek_df, hek_gdf, parcels_deep]

<h1 style="text-align:right;font-size:125%">
:איחוד עם שכבת תיקוני 97ב
<h1 style="text-align:right;font-size:100%">
הוספת רשומות שכבת 97ב אל שכבת ניהול תהליכים שנוצרה בסעיף קודם
<h1 style="text-align:right;font-size:100%">
סידור שמות של שדות
<h1 style="text-align:right;font-size:100%">
מילוי ערכים חסרים והגדרת סוג השדות (במידת הצורך)    
<h1 style="text-align:right;font-size:100%">
מחיקת שדות שאינם רלוונטיים יותר       
<h1 style="text-align:right;font-size:100%">
מיון לפי מספר פניה ותאריך בקשה            
<h1 style="text-align:right;font-size:100%">
הסרת כפילויות                

In [ ]:
appended = pd.concat([hek_gdf_dissolve, amendment97B_df], ignore_index=True)\
             .rename(columns={'תאריך בקשה'               : 'requestDate',
                              'תאריך אישור הממונה על המרשם' : 'appovalDate',
                              'תאריך אישור הרשם'          : 'registrationDate'})


appended['MISPAR_PNIA'].fillna(hek_gdf_dissolve['מספר פניה'], inplace=True)
appended['MISPAR_PNIA'] = appended['MISPAR_PNIA'].astype(int)


appended['PARCELS'].fillna(hek_gdf_dissolve['חלקות'], inplace=True)


appended['GUSH_NUM'].fillna(hek_gdf_dissolve['BlockNumber'], inplace=True)
appended['GUSH_NUM'] = appended['GUSH_NUM'].astype(int)


appended['GUSH_SUFFIX'].fillna(hek_gdf_dissolve['SubBlockNumber'], inplace=True)
appended['GUSH_SUFFIX'].fillna(0, inplace = True)
appended['GUSH_SUFFIX'] = appended['GUSH_SUFFIX'].astype(int)


appended['BlockName'] = appended['GUSH_NUM'].astype(str) + '/' + appended['GUSH_SUFFIX'].astype(str)


appended.drop(columns=['מספר פניה', 'BlockNumber', 'SubBlockNumber'], inplace=True)
appended.sort_values(['MISPAR_PNIA', 'requestDate'], inplace=True)
appended.drop_duplicates(subset='MISPAR_PNIA', keep = 'first', inplace=True)


del hek_gdf_dissolve

<h1 style="text-align:right;font-size:100%">
איחוד עם טבלת גושים לקבלת מזהה גוש
<h1 style="text-align:right;font-size:100%">
תיקוני 97ב' שלא נמצא להם גוש יוסרו מהנתונים (בהחלטת אורית ודני)

In [ ]:
appended = appended.merge(Blocks, left_on = 'BlockName', right_on = 'Name', how = 'left')\
                   .rename(columns = {'GlobalID' : 'BlockUniqueID'})

appended = appended[appended['BlockUniqueID'].notna()]

del Blocks

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
יצירת שכבה חדשה בהתאם לאפיון ✨

In [ ]:
print('   Defining new Layer')

In [ ]:
new_columns = np.dtype([
                        ("geometry", gpd.array.GeometryDtype),       
                       #("GlobalID", str),
                        ("ProcessName", str),
                        ("ProcessType", int),
                        ("Status", int),
                        ("BlockUniqueID", str),
                        ("GeodeticNetwork", int),
                       #("SurveyorLicenseID", int),
                        ("DataSource", int),
                       #("PlanName", str),  
                        ("ORACLE_ID", int),
                       #("Shape_Length", float),
                       #("Shape_Area", float),
                        ])


b97_new = gpd.GeoDataFrame(data = np.empty(0, dtype = new_columns),
                           geometry = 'geometry',
                           crs = 2039)

del new_columns

<h1 style="text-align:right;font-size:100%">
איכלוס שדות

In [ ]:
b97_new['geometry'] = appended['geometry']


b97_new['ORACLE_ID'] = appended['MISPAR_PNIA']


b97_new['ProcessName'] = appended['MISPAR_PNIA'].astype('str') + '/' + appended['BlockName'].astype('str')


b97_new['BlockUniqueID'] = appended['BlockUniqueID']


b97_new['ProcessType'] = 6 #תיקון 97ב


b97_new['GeodeticNetwork'] = 0 #רשת לא ידועה


b97_new['Status'] = appended['status'].fillna(14)  #בביצוע
b97_new['Status'] = b97_new['Status'].astype('Int64')


#b97_new['PlanName'] = None


#b97_new['SurveyorLicenseID'] = None


b97_new['DataSource'] = np.where(~b97_new['geometry'].isna(), 5, 3)  #תהליך עם גיאומטריה מגיע מקובץ שרטוט, אחרת מגיע מדיגיטציה
b97_new['DataSource'] = b97_new['DataSource'].fillna(3)
b97_new['DataSource'] = b97_new['DataSource'].astype(int)

In [ ]:
b97_new.sort_values('ProcessName', inplace=True)

b97_new = b97_new.to_crs(2039)

del appended

<h1 style="text-align:right; font-size:150%; color:#1c6ce6;"> 
שמירת תוצאות 💾
<h1 style="text-align:right; font-size:100%;">

<h1 style="text-align:right; font-size:100%;">     
פיצול התוצאות לפי סוגי גיאומטריה שונים (פעולה טכנית כי לא ניתן לכתוב קובץ סוגי גיאומטריה שונים באותה שכבה)    
<h1 style="text-align:right; font-size:100%;"> 
שמירת שכבה גיאוגרפית 

In [ ]:
print('   Exporting results')

In [ ]:
Polygons      = b97_new[(~b97_new['geometry'].isna()) & (b97_new['geometry'].geom_type == 'Polygon')]

MultiPolygons = b97_new[(~b97_new['geometry'].isna()) & (b97_new['geometry'].geom_type == 'MultiPolygon')]

EmptyGeometry = b97_new[b97_new['geometry'].isna()]

In [ ]:
Polygons.to_file(CNFG.Results, layer = 'PT6', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'w')
MultiPolygons.to_file(CNFG.Results, layer = 'PT6', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')
EmptyGeometry.to_file(CNFG.Results, layer = 'PT6', driver = 'OpenFileGDB', engine = 'pyogrio', mode = 'a')